In [ ]:
# Modelo de árbol de decisión con aprendizaje que permite clasificar a los vehículos
# en baratos y caros usando la mediana de los precios como punto de corte
# , utilizando el archivo 'ML_cars.csv'.
# By Rafael Balestrini
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import entropy
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from misFunciones import impurezaGini
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../datasource/DatasetForML.csv')
df.columns

### Entropía

In [ ]:
# Calcular la entropía de cada variable en el DataFrame
entropias = {}
for columna in df.columns:
    entropias[columna] = entropy(df[columna].value_counts(normalize=True), base=2)

# Ordenar las entropías de mayor a menor
entropiasOrdenadas = sorted(entropias.items(), key = lambda x: x[1], reverse = True)

# Imprimir los resultados ordenados
for variable, entropia in entropiasOrdenadas:
    print(f'{variable} -> entropía: {entropia}')

### Interpretación de la Impureza GINI
La impureza de gini se puede definir informalmente como la probabilidad de clasificar incorrectamente a una observación dado un conjunto de datos. A menor impureza mejor es el clasificador. La mayor impureza es 0.5 por la tanto el peor clasificador.

In [ ]:
variables = df.drop('price_category', axis = 1)

In [ ]:
ginis = {}
for variable in variables:
    ginis[variable] = impurezaGini(variable, "price_category", df)

# Ordenar las impurezas gini de menor a mayor
ginisOrdenadas = sorted(ginis.items(), key = lambda x: x[1])

# Imprimir los resultados ordenados
for variable, gini in ginisOrdenadas:
    print(variable, '-> impureza: %0.4f' % gini)

### Variables predictoras y la variable objetivo

In [ ]:
# 'X' es todo el dataframe original pero sin la columna 'price_category'. Es la matriz 
# con las variables independientes o predictoras o también llamadas características
# X = df.drop('price_category', axis = 1) # (205, 61)
# X = df[['cylindernumber']]
X = df[['curbweight', 'highwaympg', 'citympg']]

In [ ]:

# 'y' es un n-arreglo unidimensional de Pandas que contiene las
# variables dependientes o etiquetas o también llamadas objetivos
                         # 0    103    Barato
y = df['price_category'] # 1    102    Caro
                         # (205,) dtype: int64

In [ ]:
# Profundidad máxima del árbol
proMax = 4

### Datos de entrenamiento y pruebas

In [ ]:
#criterio = 'entropy'
criterio = 'gini'

In [ ]:
# Dividimos la matriz X en conjuntos de entrenamiento y prueba
# X_train es un dataframe con el 80% de X para entrenar
# X_test es un dataframe con el 20% de X para las pruebas
# y_train es una serie con el 80% de y para entrenar
# y_test es una serie con el 20% de y para las pruebas
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Creamos el modelo de árbol de decisión
modeloArbol = DecisionTreeClassifier(criterion = criterio, max_depth = proMax)  # Profundidad máxima 
#modeloArbol = DecisionTreeClassifier()  # Profundidad máxima todas

# Ponemos a entrenar al modelo
modeloArbol.fit(XTrain, yTrain)

# Aplicamos la clasificación a los datos de prueba
yPred = modeloArbol.predict(XTest)

In [ ]:
# Evaluamos el rendimiento del modelo
accuracy = accuracy_score(yTest, yPred)
accuracy

In [ ]:
# Graficamos el árbol
plt.figure(figsize=(15, 10))
plot_tree(modeloArbol, filled=True, feature_names=X.columns, class_names=['barato', 'caro'])
plt.show()

### Calibración de hiperparámetro

In [ ]:
# Crear un DataFrame vacío para almacenar los resultados
accuracies = pd.DataFrame(columns = [
    'Variable', 'maxDepth 1', 'maxDepth 2', 'maxDepth 3', 'maxDepth 4',
    'maxDepth 5', 'maxDepth 6', 'Metodo'])

variables = ['curbweight', 'carlength', 'horsepower', 'carheight', 'wheelbase', 'price_category']

for variable in variables:
    maxDepths = []
    for depth in range(accuracies.shape[1] - 2):
        X = df[[variable]]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        # Creamos el modelo de árbol de decisión
        modeloArbol = DecisionTreeClassifier(criterion = criterio, max_depth = depth + 1)  # Profundidad máxima
        
        # Ponemos a entrenar al modelo        
        modeloArbol.fit(X_train, y_train)
        
        # Aplicamos la clasificación a los datos de prueba
        y_pred = modeloArbol.predict(X_test)

        # Evaluamos el rendimiento del modelo
        accuracy = accuracy_score(y_test, y_pred)
        #print(f'accuracy={accuracy}')
        maxDepths.append(accuracy) 

        if depth == proMax - 1:
            # Graficamos el árbol
            plt.figure(figsize=(15, 10))
            plot_tree(modeloArbol, filled=True, feature_names=X.columns, class_names=['barato', 'caro'])
            plt.show()

    #print(variable, maxDepths)
    # Agregar el resultado al DataFrame
    accuracies.loc[len(accuracies)] = [variable, maxDepths[0], maxDepths[1], maxDepths[2],
                                       maxDepths[3], maxDepths[4], maxDepths[5], 'gini']  

# Mostrar el DataFrame resultante
accuracies

In [ ]:
# Evaluamos el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)